In [0]:
%pip install pydicom==3.0.1

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0.0/2.4 MB ? eta -:--:--
   ━━━━━━━╸━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0.5/2.4 MB 14.5 MB/s eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 39.5 MB/s eta 0:00:00
Note: you may need to restart the kernel using %restart_python or dbutils.library.restartPython() to use updated packages.


In [0]:
import logging
logger = logging.getLogger("zipdcm")
logger.setLevel(logging.DEBUG)
formatter = logging.Formatter('%(asctime)s - %(filename)s:%(lineno)s - %(levelname)s - %(message)s')
# Create a handler (e.g., StreamHandler for console output)
handler = logging.StreamHandler()
handler.setFormatter(formatter)

# Add the handler to the logger
logger.addHandler(handler)

### How many zip files 
do we have on `/Volumes/hls_radiology/tcia/downloads/tciaDownload`

In [0]:
# how many zips do we have?
df = (
    spark.read.format("binaryFile")
        .option("pathGlobFilter", "*.zip")
        .option("recursiveFileLookup", "true")
        .load("/Volumes/hls_radiology/tcia/downloads/tciaDownload").drop('content')
    )
df.count()

1416

In [0]:
table_name = "douglas_moore.pixels.zipdcm_test"

In [0]:
%sql
DROP TABLE IF EXISTS IDENTIFIER(:table_name);

In [0]:
from zip_dcm_ds import ZipDCMDataSource

import time
start_time = time.time()
total_worker_cores = 16 


# Add our custom Python DataSource for DICMO files storaged in a Zip Archive
spark.dataSource.register(ZipDCMDataSource)

def test_scale(spark, numPartitions=64):
    df = (
        spark.read
          .option('numPartitions',f'{numPartitions}')
          .format("zipdcm")
          .load("/Volumes/hls_radiology/tcia/downloads/tciaDownload") #1416 zip files
    )
    df.write.format('delta').mode('overwrite').option("mergeSchema", "true").saveAsTable(table_name)

test_scale(spark, 1024)


end_time = time.time()
operation_time = end_time - start_time
logger.info(f"Operation time: {operation_time} seconds and {operation_time * total_worker_cores} core-seconds")

2025-06-01 19:11:56,803 - command-2588019255111356-2308021588:25 - INFO - Operation time: 247.52023124694824 seconds and 3960.323699951172 core-seconds


## Summary stats

In [0]:
%sql
select
count(distinct path) paths,
count(distinct split(path,'::')[0]) zipfiles,
count(distinct rowid) rowids, count(1) records
from IDENTIFIER(:table_name)

paths,zipfiles,rowids,records
107158,1416,3451,107158


In [0]:
print(sc.show_profiles())

None


In [0]:
%sql
select zipfile,count(distinct dcmfile) 
from douglas_moore.pixels.zipdcm_test
group by zipfile 
order by 2 desc

---------------------------------------------------------------------------
AnalysisException                         Traceback (most recent call last)
File <command-2588019255111359>, line 1
----> 1 get_ipython().run_cell_magic('sql', '', 'select zipfile,count(distinct dcmfile) \nfrom douglas_moore.pixels.zipdcm_test\ngroup by zipfile \norder by 2 desc\n')

File /databricks/python/lib/python3.12/site-packages/IPython/core/interactiveshell.py:2541, in InteractiveShell.run_cell_magic(self, magic_name, line, cell)
   2539 with self.builtin_trap:
   2540     args = (magic_arg_s, cell)
-> 2541     result = fn(*args, **kwargs)
   2543 # The code below prevents the output from being displayed
   2544 # when using magics with decorator @output_can_be_silenced
   2545 # when the last Python token in the expression is a ';'.
   2546 if getattr(fn, magic.MAGIC_OUTPUT_CAN_BE_SILENCED, False):

File /databricks/python_shell/lib/dbruntime/sql_magic/sql_magic.py:130, in SqlMagic.sql(self, line, cell

## Sample data records

In [0]:
%sql
select * 
from douglas_moore.pixels.zipdcm_test
limit 100;